# Titanic Survival Prediction using Neural Networks

This notebook implements a deep learning approach to the Titanic survival prediction problem using TensorFlow.

## Approach:
1. Data preprocessing with feature engineering
2. Neural network architecture with regularization
3. Training with class weights and callbacks
4. Model evaluation and submission generation

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from datetime import datetime
from keras import callbacks, layers, models

## 1. Data Preprocessing

In [ ]:
# Load data
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# Select features
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

# Prepare features
X_test = pd.get_dummies(test[features])
X = pd.get_dummies(train[features])
y = train['Survived']

# Handle missing values
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X)
X = pd.DataFrame(imp.transform(X), columns=X.columns)
X_test = pd.DataFrame(imp.transform(X_test), columns=X_test.columns)

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

## 2. Model Architecture

In [ ]:
def create_model(input_dim):
    """Create a neural network model with regularization techniques.
    
    Args:
        input_dim: Number of input features
        
    Returns:
        Compiled Keras model
    """
    # L2 regularization to prevent overfitting
    regularizer = tf.keras.regularizers.l2(0.01)
    
    model = models.Sequential([
        # Input layer
        layers.Dense(32, kernel_regularizer=regularizer, input_shape=(input_dim,)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.4),
        
        # Hidden layer 1
        layers.Dense(24, kernel_regularizer=regularizer),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.4),
        
        # Hidden layer 2
        layers.Dense(16, kernel_regularizer=regularizer),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.4),
        
        # Output layer
        layers.Dense(1, activation='sigmoid')
    ])
    
    # Optimizer with momentum
    optimizer = tf.keras.optimizers.SGD(
        learning_rate=0.001,
        momentum=0.9,
        nesterov=True
    )
    
    # Compile model with multiple metrics
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            tf.keras.metrics.AUC(name='auc'),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall')
        ]
    )
    
    return model

## 3. Model Training

In [ ]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices(
    (X_train_scaled, y_train.values)
).shuffle(1000).batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices(
    (X_val_scaled, y_val.values)
).batch(32)

# Initialize model
model = create_model(X_train_scaled.shape[1])

# Setup callbacks
callbacks_list = [
    # Early stopping to prevent overfitting
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        min_delta=0.001
    ),
    
    # Reduce learning rate when training plateaus
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=0.00001,
        verbose=1
    ),
    
    # Save best model
    callbacks.ModelCheckpoint(
        'best_model.keras',
        monitor='val_auc',
        save_best_only=True,
        mode='max'
    )
]

# Calculate class weights
class_weights = {
    0: 1.0,
    1: (y_train == 0).sum() / (y_train == 1).sum()
}

# Train model
history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=val_dataset,
    callbacks=callbacks_list,
    class_weight=class_weights,
    verbose=1
)

## 4. Model Evaluation

In [ ]:
def plot_training_history(history):
    """Plot training metrics."""
    plt.figure(figsize=(12, 4))

    # Loss plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # AUC plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['auc'], label='Training AUC')
    plt.plot(history.history['val_auc'], label='Validation AUC')
    plt.title('Model AUC')
    plt.xlabel('Epoch')
    plt.ylabel('AUC')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Plot training history
plot_training_history(history)

# Plot ROC curve
y_pred = model.predict(X_val_scaled)
fpr, tpr, _ = roc_curve(y_val, y_pred)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2,
         label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

## 5. Generate Submission

In [ ]:
# Generate predictions
test_pred = (model.predict(X_test_scaled) > 0.5).astype(int)

# Create submission file
submission = pd.DataFrame({
    'PassengerId': test.PassengerId,
    'Survived': test_pred.flatten()
})

# Save submission
submission.to_csv('submission.csv', index=False)
print("Submission file created successfully!")

# Display first few predictions
submission.head()